In [1]:
# Import dependencies
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
from collections import Counter
import psycopg2
from psycopg2 import Error
from config import DB_PASS

## Connect to and Load from Database

In [2]:
# Connection parameters
param_dic = {
    "host"      : "ec2-34-239-241-121.compute-1.amazonaws.com",
    "database"  : "dfm419pf436p66",
    "user"      : "eulpledavzofae",
    "password"  : DB_PASS
}
def connect(params_dic):
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params_dic)
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        sys.exit(1) 
    print("Connection successful")
    return conn

In [3]:
def postgresql_to_dataframe(conn, select_query, column_names):
    """
    Tranform a SELECT query into a pandas dataframe
    """
    cursor = conn.cursor()
    try:
        cursor.execute(select_query)
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        cursor.close()
        return 1
    
    #list of tupples
    tupples = cursor.fetchall()
    cursor.close()
    
    # turn it into a pandas dataframe
    heart_dp = pd.DataFrame(tupples, columns=column_names)
    return heart_dp

In [4]:
# Connect to the database
conn = connect(param_dic)
column_names = ["Pt_ID", "Age", "Sex", "ChestPainType", "RestingBP", "Cholesterol", "FastingBS","RestingEKG","MaxHR","ExerciseAngina","OldPeak","ST_Slope", "HeartDisease"]
# Execute the "SELECT *" query
heart_disease_df = postgresql_to_dataframe(conn, "select * from heart_dp_train", column_names)
heart_disease_df.head()

Connecting to the PostgreSQL database...
Connection successful


,Pt_ID,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingEKG,MaxHR,ExerciseAngina,OldPeak,ST_Slope,HeartDisease
0,25047,74,M,NAP,138,0,0,Normal,116,N,0.2,Up,Absence
1,25177,60,M,ASY,130,186,1,ST,140,Y,0.5,Flat,Presence
2,25498,40,M,ASY,125,0,1,Normal,165,N,0,Flat,Presence
3,25189,69,M,ASY,140,208,0,ST,140,Y,2,Flat,Presence
4,25434,62,M,ASY,158,210,1,Normal,112,Y,3,Down,Presence


## Export DataFrame to csv file for Tableau

## Pre-Processing

In [5]:
# Confirm entry count
heart_disease_df['HeartDisease'].value_counts()

Presence    508
Absence     410
Name: HeartDisease, dtype: int64

In [6]:
# Encode object variables with Scikit Learn
object_columns = heart_disease_df.dtypes[heart_disease_df.dtypes == "object"].index.tolist()
le = LabelEncoder()
encoded_df = heart_disease_df.copy()

for column in object_columns:
    encoded_df[column] = le.fit_transform(encoded_df[column])
    
encoded_df.head()

,Pt_ID,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingEKG,MaxHR,ExerciseAngina,OldPeak,ST_Slope,HeartDisease
0,25047,74,1,2,138,0,0,1,116,0,12,2,0
1,25177,60,1,0,130,186,1,2,140,1,15,1,1
2,25498,40,1,0,125,0,1,1,165,0,10,1,1
3,25189,69,1,0,140,208,0,2,140,1,30,1,1
4,25434,62,1,0,158,210,1,1,112,1,39,0,1


In [7]:
# Define the target and features
y = encoded_df['HeartDisease'].ravel()

X = encoded_df.drop('HeartDisease', axis = 1)

In [8]:
# Split in to test and train sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 22)

In [9]:
# Create StandardScaler instance
scaler = StandardScaler()

# Fit the scaler with the training data
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Create and Fit Random Forest Model

In [10]:
# Create random forest classifier
model = RandomForestClassifier(n_estimators = 128, random_state = 20)

In [11]:
# Fit the model
model = model.fit(X_train_scaled, y_train)

## Make Predictions

In [12]:
# Make predictions using the testing data
predictions = model.predict(X_test_scaled)

## Assess Accuracy

In [13]:
# Calculate confusion matrix
matrix = confusion_matrix(y_test, predictions)
matrix

array([[ 89,  11],
       [ 21, 109]], dtype=int64)

In [14]:
# Calculate accuracy score
acc_score = accuracy_score(y_test, predictions)
print(f"Accuracy score: {acc_score}")

Accuracy score: 0.8608695652173913


In [16]:
class_report = classification_report(y_test, predictions)
print(class_report)

              precision    recall  f1-score   support

           0       0.81      0.89      0.85       100
           1       0.91      0.84      0.87       130

    accuracy                           0.86       230
   macro avg       0.86      0.86      0.86       230
weighted avg       0.87      0.86      0.86       230



## Feature Importances

In [17]:
# Calculate feature importance
importances = model.feature_importances_
importances

array([0.07268628, 0.06633403, 0.02980734, 0.12155114, 0.05884815,
       0.09755969, 0.0304638 , 0.02778631, 0.11061294, 0.11089429,
       0.09509723, 0.17835879])

In [18]:
# Sort the features by their importance.
sorted(zip(model.feature_importances_, X.columns), reverse=True)

[(0.17835879467563698, 'ST_Slope'),
 (0.12155113783065993, 'ChestPainType'),
 (0.11089428808631394, 'ExerciseAngina'),
 (0.11061294039219947, 'MaxHR'),
 (0.09755969327652136, 'Cholesterol'),
 (0.09509722739553886, 'OldPeak'),
 (0.07268628032510893, 'Pt_ID'),
 (0.06633403116482448, 'Age'),
 (0.05884815089442937, 'RestingBP'),
 (0.030463804247622704, 'FastingBS'),
 (0.029807344708313088, 'Sex'),
 (0.027786307002830957, 'RestingEKG')]

## Create DataFrame Predictions vs. Actual

In [29]:
compare_dict = {
    'Prediction': predictions,
    'Actual': y_test
}

In [32]:
compare_df = pd.DataFrame.from_dict(compare_dict)
compare_df

,Prediction,Actual
0,1,1
1,0,1
2,0,1
3,0,1
4,1,1
...,...,...
225,1,1
226,0,0
227,1,0
228,1,1


In [33]:
path = "../Predictions_Comparison.csv"
compare_df.to_csv(path)